In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
from matplotlib import pyplot as plt

import scanpyHelpers as scH

In [ ]:
writeDir = "write"
tissue = "Liver"

species1 = "human"
adataFile1 = f'{writeDir}/{species1}{tissue}_QC.h5ad'

species2 = "mouse"
adataFile2 = f'{writeDir}/{species2}{tissue}_QC.h5ad'

mergedAdataFileQC = f'{writeDir}/{tissue}{species1}{species2}_QC.h5ad'
mergedAdataFile = f'{writeDir}/{tissue}{species1}{species2}.h5ad'


homologyFile = "data/hgncHM_121.csv"

In [ ]:
homology = pd.read_table(homologyFile, sep=",")
homology

In [ ]:
adata1 = sc.read(adataFile1)
adata1.layers["counts"] = adata1.X.copy()
adata1

In [ ]:
sc.pp.normalize_total(adata1, target_sum=1e4)#,exclude_highly_expressed=True)#
sc.pp.log1p(adata1)

In [ ]:
adata2 = sc.read(adataFile2)
adata2.layers["counts"] = adata2.X.copy()
adata2

In [ ]:
sc.pp.normalize_total(adata2, target_sum=1e4)#,exclude_highly_expressed=True)#
sc.pp.log1p(adata2)

In [ ]:
genes1 = np.array(adata1.var_names, dtype=object)
genes2 = np.array(adata2.var_names, dtype=object)
genes1, genes2 = scH.getOverlapGenes(genes1, genes2, homology, species1, species2)

In [ ]:
print(f"{len(genes1)} {len(genes2)} {len(set(genes1).intersection(set(genes2)))}")

In [ ]:
adata1.var_names = genes1
adata2.var_names = genes2

In [ ]:
adata = ad.concat([adata1, adata2], join="outer", label="species", keys=[species1, species2], fill_value=0)
adata.obs_names_make_unique()

In [ ]:
adata.obs

In [ ]:
adata.obs = adata.obs[["species","CellType"]]
adata

In [ ]:
mergedAdataFileQC

In [ ]:
adata.write(mergedAdataFileQC)

In [ ]:
adata.var

In [ ]:
batchKey = "species"
cellTypeKey = "CellType"

In [ ]:
sc.pp.highly_variable_genes(adata, batch_key=batchKey)

In [ ]:
minMean = 0.05
maxMean = 2.9
minDisp = 0.25

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

means = adata.var[["means"]][adata.var[["means"]] > np.exp(-14)]#adata.var[["means"]],
axs[0].hist(np.log(means), bins=100)#, log=True),
axs[0].axvline(np.log(minMean), color='k', linestyle='dashed', linewidth=1)
axs[0].axvline(np.log(maxMean), color='k', linestyle='dashed', linewidth=1)
axs[0].set_title('Gene means counts')
axs[0].set_xlabel('means')
axs[0].set_ylabel('counts')

dispNorm = adata.var[["dispersions_norm"]][adata.var[["dispersions_norm"]] > np.exp(-8)]#adata.var[["means"]],
axs[1].hist(np.log(dispNorm), bins=100)#, log=True),
axs[1].axvline(np.log(minDisp), color='k', linestyle='dashed', linewidth=1)
axs[1].set_title('Gene dispersions counts')
axs[1].set_xlabel('dispersions')
axs[1].set_ylabel('counts')

sc.pp.highly_variable_genes(adata, min_disp=minDisp, min_mean=minMean, max_mean=maxMean, batch_key=batchKey)
print(sum(adata.var.highly_variable))
print(sum(adata.var.highly_variable_intersection))

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.var[adata.var.highly_variable]

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-') 

In [ ]:
adata.raw = adata

In [ ]:
#adata = adata[:, adata.var.highly_variable]
adata = adata[:, np.logical_and(adata.var.highly_variable, np.logical_not(adata.var.mt))]

In [ ]:
#sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
#sc.pp.scale(adata, max_value=5)

In [ ]:
sc.tl.pca(adata, n_comps = 100, svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs = 100, log=True)

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs = 100)

In [ ]:
sc.pl.pca(adata, color=["CellType"])

In [ ]:
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=40)

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution=0.2)

In [ ]:
adata

In [ ]:
sc.pl.umap(adata, color=["leiden",cellTypeKey,batchKey],ncols=2)

In [ ]:
cellTypeDict = {'B':"B", 
                'Central_venous_LSECs':"LSEC", 
                'Cholangiocytes':"Cholang", 
                'DCs':"DC",
                'Endothelial cells':"Endo", 
                'Erythroid_Cells':"Eryth",
                'Hepatic progenitor cells':"HepaticProg",
                'Hepatic_Stellate_Cells':"Stellate",
                'Hepatocyte_1':"Hepato",
                'Hepatocyte_2':"Hepato",
                'Hepatocyte_3':"Hepato",
                'Hepatocyte_4':"Hepato",
                'Hepatocyte_5':"Hepato",
                'Hepatocyte_6':"Hepato",
                'Hepatocytes':"Hepato",
                'Inflammatory_Macrophage':"Macro",
                'Kupffer cells':"Kupffer", 
                'Mature_B_Cells':"B", 
                'Monocyte/Monocyte derived macrophage':"Mono",
                'Myofibroblasts':"Myofib",
                'NK':"NK",
                'NK-like_Cells':"NK", 
                'Neutrophils':"Neutro",
                'Non-inflammatory_Macrophage':"Macro",
                'Periportal_LSECs':"LSEC", 
                'Plasma_Cells':"Plasma",
                'Portal_endothelial_Cells':"Endo", 
                'Stellate cells':"Stellate",
                'T':"T", 
                'alpha-beta_T_Cells':"T",
                'gamma-delta_T_Cells_1':"T",
                'gamma-delta_T_Cells_2':"T",
                'pDCs':"DC"}
adata.obs["simple"] = [cellTypeDict[c] for c in adata.obs["CellType"]]

In [ ]:
sc.pl.umap(adata, color=["CellType"],ncols=1, legend_loc="on data")

In [ ]:
sc.pl.umap(adata, color=["simple"],ncols=1, legend_loc="on data")

In [ ]:
mergedAdataFile

In [ ]:
adata.write(mergedAdataFile)

In [ ]:
adata

In [ ]:
adata.var = adata.var[["highly_variable_nbatches"]]
adata.obsm = None
adata.varm = None
adata.obsp = None

In [ ]:
adata